In [1]:
from Core.Preprocess import *
from Core.CryptoCompareAPI import *
from Core.config_reader import config_reader

In [84]:
import pandas as pd
import calendar

## Download Data

In [3]:
reader = config_reader('/Users/admin/PycharmProjects/Market_Prediction_Using_News_and_Indicators/config.yml')

crypto_currency_list = reader.read_config("CRYPTO_CURRENCIES_LIST")
exchange = reader.read_config("EXCHANGE")
currency = reader.read_config("CURRENCY")

In [218]:
df_crypto_list = []

In [219]:
 for crypto in crypto_currency_list:
    raw_data = get_historical_price_hour_from(
        coin=crypto,
        currency=currency,
        exchange=exchange,
        toTs=int(calendar.timegm(datetime.datetime(2022, 5, 1, 0, 0, 0).timetuple())),
        fromTs=int(calendar.timegm(datetime.datetime(2022, 2, 1, 0, 0, 0).timetuple()))
    )
    data = Preprocess.get_from_crypto_compare_hourly_price_data_to_pandas(raw_data)
    df_crypto_list.append(data)

## Preprocessing

In [220]:
df_crypto_list[0]

,close,high,low,open,volumefrom,volumeto,conversionType,conversionSymbol,datetimes
time,,,,,,,,,
2022-02-01 00:00:00,38342.36,38627.35,38276.43,38466.90,1058.43,4.064914e+07,force_direct,,2022-02-01
2022-02-01 01:00:00,38238.19,38407.00,38175.86,38342.36,777.86,2.978374e+07,force_direct,,2022-02-01
2022-02-01 02:00:00,38480.90,38506.18,38196.61,38238.19,766.84,2.941555e+07,force_direct,,2022-02-01
2022-02-01 03:00:00,38596.82,38649.90,38384.99,38480.90,946.46,3.646598e+07,force_direct,,2022-02-01
2022-02-01 04:00:00,38556.82,38644.57,38470.37,38596.82,758.93,2.926184e+07,force_direct,,2022-02-01
...,...,...,...,...,...,...,...,...,...
2022-04-30 20:00:00,38326.45,38466.83,38282.08,38466.83,676.71,2.596374e+07,force_direct,,2022-04-30
2022-04-30 21:00:00,38301.14,38370.84,38248.66,38326.45,591.08,2.264638e+07,force_direct,,2022-04-30
2022-04-30 22:00:00,38250.44,38370.00,38160.00,38301.14,1376.35,5.267822e+07,force_direct,,2022-04-30


In [221]:
for i in range(len(df_crypto_list)):
    Preprocess.rename_columns(df_crypto_list[i])

In [222]:
df_crypto_list[0]

,Close,High,Low,Open,Volume
time,,,,,
2022-02-01 00:00:00,38342.36,38627.35,38276.43,38466.90,4.064914e+07
2022-02-01 01:00:00,38238.19,38407.00,38175.86,38342.36,2.978374e+07
2022-02-01 02:00:00,38480.90,38506.18,38196.61,38238.19,2.941555e+07
2022-02-01 03:00:00,38596.82,38649.90,38384.99,38480.90,3.646598e+07
2022-02-01 04:00:00,38556.82,38644.57,38470.37,38596.82,2.926184e+07
...,...,...,...,...,...
2022-04-30 20:00:00,38326.45,38466.83,38282.08,38466.83,2.596374e+07
2022-04-30 21:00:00,38301.14,38370.84,38248.66,38326.45,2.264638e+07
2022-04-30 22:00:00,38250.44,38370.00,38160.00,38301.14,5.267822e+07


In [223]:
def sign(x):
    return x/abs(x)

target_list = []

for df in df_crypto_list:
    target_list.append(sign(df_crypto_list[0]['Close'].shift(-24) - df_crypto_list[0]['Close'])[90:-24])

In [224]:
target_list[0]

time
2022-02-04 18:00:00    1.0
2022-02-04 19:00:00    1.0
2022-02-04 20:00:00    1.0
2022-02-04 21:00:00    1.0
2022-02-04 22:00:00    1.0
                      ... 
2022-04-29 20:00:00   -1.0
2022-04-29 21:00:00   -1.0
2022-04-29 22:00:00   -1.0
2022-04-29 23:00:00   -1.0
2022-04-30 00:00:00   -1.0
Name: Close, Length: 2023, dtype: float64

In [225]:
new_df_crypto_list = list(map(Preprocess.compute_all_indicators, df_crypto_list))

In [226]:
for i in range(len(new_df_crypto_list)):
    new_df_crypto_list[i].drop(columns=["Close", "High", "Low", "Open", "Volume"], inplace=True)
    new_df_crypto_list[i] = new_df_crypto_list[i].iloc[90:-24]

In [228]:
for i in range(len(df_crypto_list)):
    df_crypto_list[i].drop(columns=["UpperBB", "LowerBB"], inplace = True)
    df_crypto_list[i] = df_crypto_list[i].iloc[90:-24]

In [229]:
df_crypto_list[0]

,Close,High,Low,Open,Volume
time,,,,,
2022-02-04 18:00:00,40469.27,40662.01,40278.79,40520.46,8.461738e+07
2022-02-04 19:00:00,40727.91,40877.00,40366.99,40469.27,9.520640e+07
2022-02-04 20:00:00,40588.00,40884.96,40456.25,40727.91,1.041768e+08
2022-02-04 21:00:00,40624.15,40631.89,40426.23,40588.00,6.104136e+07
2022-02-04 22:00:00,40641.74,40666.98,40382.84,40624.15,4.642513e+07
...,...,...,...,...,...
2022-04-29 20:00:00,38564.44,38584.00,38310.90,38331.54,7.666395e+07
2022-04-29 21:00:00,38560.15,38649.99,38450.47,38564.44,3.069227e+07
2022-04-29 22:00:00,38608.63,38635.29,38430.03,38560.15,2.538127e+07


In [230]:
new_df_crypto_list[0]

,UpperBB,LowerBB,DEMA,EMA,HT_TRENDLINE,KAMA,MA,MIDPOINT,MIDPRICE,SAR,...,STOCHRSI FASTD,TRIX,ULTOSC,WILLR,AD,ADOSC,OBV,TRANGE,ATR,NATR
time,,,,,,,,,,,,,,,,,,,,,
2022-02-04 18:00:00,40353.302329,35918.871671,38724.151122,38100.630020,37434.617702,38402.621527,37660.167667,38970.930,38961.010,36250.00,...,98.476603,-0.011691,64.119127,-5.665491,4.163835e+08,2.232880e+08,8.408365e+07,383.22,520.992874,0.003181
2022-02-04 19:00:00,40769.482460,35844.321540,39011.992214,38270.131955,37518.339136,38739.630392,37798.262000,39074.655,39083.600,36250.00,...,98.476603,-0.002145,64.504258,-4.156630,4.559270e+08,2.169142e+08,1.792901e+08,510.01,520.208383,0.003136
2022-02-04 20:00:00,41079.282361,35875.268639,39253.562275,38419.671829,37598.399669,38997.767312,37934.338333,39074.655,39087.580,36250.00,...,65.143270,0.007877,67.242522,-8.260913,4.157808e+08,1.818135e+08,7.511321e+07,428.71,513.672784,0.003118
2022-02-04 21:00:00,41343.475235,35956.630765,39475.035920,38561.896227,37675.517391,39207.938299,38058.141667,39074.655,39087.580,36250.00,...,35.991913,0.018190,67.727990,-7.255283,4.722276e+08,1.688860e+08,1.361546e+08,205.66,491.671871,0.002979
2022-02-04 22:00:00,41573.542988,36058.112012,39675.833623,38696.079696,37766.415297,39423.053828,38189.966333,39074.655,39087.580,36250.00,...,6.690602,0.028623,65.822548,-6.765959,5.104049e+08,1.603915e+08,1.825797e+08,284.14,476.848166,0.002887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-29 20:00:00,39983.202962,38203.066038,38862.692454,39110.594185,39333.395743,39231.793070,39355.197667,38970.840,38904.560,34322.28,...,40.017682,0.000117,44.058563,-73.309940,1.911979e+09,-2.728047e+07,-9.507778e+08,273.10,370.641565,0.002492
2022-04-29 21:00:00,39880.959476,38179.276524,38809.952189,39075.081657,39313.450401,39212.194712,39325.935667,38870.880,38904.560,34322.28,...,66.280453,-0.002887,45.451942,-73.603953,1.915031e+09,-1.402109e+07,-9.814700e+08,199.52,358.418596,0.002411
2022-04-29 22:00:00,39822.051152,38146.835848,38768.811531,39044.988002,39297.106258,39178.867493,39282.908667,38804.295,38792.615,34322.28,...,99.613787,-0.005823,48.213134,-64.894797,1.933819e+09,-1.344152e+06,-9.560888e+08,205.26,347.478696,0.002331


## Build Models

In [281]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

### MLP

In [280]:
from sklearn.neural_network import MLPClassifier

#### BTC

#### No indicators

In [282]:
X_btc = df_crypto_list[0]
y_btc = target_list[0]

X_btc_train, X_btc_test, y_btc_train, y_btc_test = train_test_split(X_btc, y_btc, test_size=0.33, random_state=42)

In [240]:
mlp_model_btc = MLPClassifier(solver='adam', alpha=0.0002, hidden_layer_sizes=(71,))

mlp_model_btc.fit(X_btc_train, y_btc_train)

MLPClassifier(alpha=0.0002, hidden_layer_sizes=(71,))

In [241]:
y_predict_btc = mlp_model_btc.predict(X_btc_test)

In [242]:
precision = precision_score(y_btc_test, y_predict_btc)
recall = recall_score(y_btc_test, y_predict_btc)
accuracy = accuracy_score(y_btc_test, y_predict_btc)
f1 = f1_score(y_btc_test, y_predict_btc)

print(f"precision = {precision}")
print(f"recall = {recall}")
print(f"accuracy = {accuracy}")
print(f"f1-score = {f1}")

precision = 0.48952095808383234
recall = 1.0
accuracy = 0.48952095808383234
f1-score = 0.657286432160804


In [310]:
folds = KFold(n_splits=2, shuffle=True, random_state=42)

precision_list = []
recall_list = []
accuracy_list = []
f1_list = []

for i, (train_index, test_index) in enumerate(folds.split(X_btc)):
    mlp_model_btc = MLPClassifier(solver='adam', alpha=0.0002, hidden_layer_sizes=(71,))
    mlp_model_btc.fit(X_btc.iloc[train_index], y_btc.iloc[train_index])
    y_predict_btc = mlp_model_btc.predict(X_btc.iloc[test_index])

    precision_list.append(precision_score(y_btc.iloc[test_index], y_predict_btc))
    recall_list.append(recall_score(y_btc.iloc[test_index], y_predict_btc))
    accuracy_list.append(accuracy_score(y_btc.iloc[test_index], y_predict_btc))
    f1_list.append(f1_score(y_btc.iloc[test_index], y_predict_btc))

precision = sum(precision_list) / len(precision_list)
recall = sum(recall_list) / len(recall_list)
accuracy = sum(accuracy_list) / len(accuracy_list)
f1 = sum(f1_list) / len(f1_list)

print(f"precision = {precision}")
print(f"recall = {recall}")
print(f"accuracy = {accuracy}")
print(f"f1-score = {f1}")

precision = 0.4839390225308171
recall = 1.0
accuracy = 0.4839390225308171
f1-score = 0.652190007976255


#### With Indicators

In [311]:
X_btc = new_df_crypto_list[0]
y_btc = target_list[0]

X_btc_train, X_btc_test, y_btc_train, y_btc_test = train_test_split(X_btc, y_btc, test_size=0.33, random_state=42)

In [278]:
mlp_model_btc = MLPClassifier(solver='adam', alpha=0.0002, hidden_layer_sizes=(71,))

mlp_model_btc.fit(X_btc_train, y_btc_train)

MLPClassifier(alpha=0.0002, hidden_layer_sizes=(71,))

In [279]:
y_predict_btc = mlp_model_btc.predict(X_btc_test)

precision = precision_score(y_btc_test, y_predict_btc)
recall = recall_score(y_btc_test, y_predict_btc)
accuracy = accuracy_score(y_btc_test, y_predict_btc)
f1 = f1_score(y_btc_test, y_predict_btc)

print(f"precision = {precision}")
print(f"recall = {recall}")
print(f"accuracy = {accuracy}")
print(f"f1-score = {f1}")

precision = 0.5098684210526315
recall = 0.9480122324159022
accuracy = 0.5284431137724551
f1-score = 0.6631016042780749


In [312]:
folds = KFold(n_splits=2, shuffle=True, random_state=42)

precision_list = []
recall_list = []
accuracy_list = []
f1_list = []

for i, (train_index, test_index) in enumerate(folds.split(X_btc)):
    mlp_model_btc = MLPClassifier(solver='adam', alpha=0.0002, hidden_layer_sizes=(71,))
    mlp_model_btc.fit(X_btc.iloc[train_index], y_btc.iloc[train_index])
    y_predict_btc = mlp_model_btc.predict(X_btc.iloc[test_index])

    precision_list.append(precision_score(y_btc.iloc[test_index], y_predict_btc))
    recall_list.append(recall_score(y_btc.iloc[test_index], y_predict_btc))
    accuracy_list.append(accuracy_score(y_btc.iloc[test_index], y_predict_btc))
    f1_list.append(f1_score(y_btc.iloc[test_index], y_predict_btc))

precision = sum(precision_list) / len(precision_list)
recall = sum(recall_list) / len(recall_list)
accuracy = sum(accuracy_list) / len(accuracy_list)
f1 = sum(f1_list) / len(f1_list)

print(f"precision = {precision}")
print(f"recall = {recall}")
print(f"accuracy = {accuracy}")
print(f"f1-score = {f1}")

precision = 0.48108065467616024
recall = 0.8375226477636116
accuracy = 0.48197935359269384
f1-score = 0.6106345400743352
